In [1]:
# Objective : Create a consistent target for training RNAseg

# Objective : Create a consistent target for training RNAseg
* Input : A spatial data with a potentially errounous nuclues and cell segmentation
* Output : A currated cell segmentation and nuclei segmentation that can be used to train RNAseg

In [1]:
# PATH
path_zarr = '/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr'

In [2]:
# IMPORT 
import spatialdata as sd

# IMPORT RNAseg
from dataset_zarr.consistency import compute_consistent_cell


/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


### Step 1 : Load the data

In [3]:
sdata = sd.read_zarr(path_zarr)
sdata

SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      └── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
with coordinate systems:
    ▸ 'microns', with elements:
        staining_z3 (Images), transcripts (Points), Cellbound1 (Shapes), DAPI (Shapes)

### Step 2 : Create a consistent target

In [4]:

# to set from the sdata content
key_shape_cell_seg = "Cellbound1"
key_shape_nuclei_seg="DAPI"
# to name for future shape that will be created in the sdata
key_cell_consistent = "Cellbound1_consistent"
key_nuclei_consistent = "DAPI_consistent"


# merge all cell and shape for the user to see the result

sdata, _ = compute_consistent_cell(
    sdata=sdata,
    key_shape_nuclei_seg=key_shape_nuclei_seg,
    key_shape_cell_seg=key_shape_cell_seg,
    key_cell_consistent=key_cell_consistent,
    key_nuclei_consistent=key_nuclei_consistent,
    image_key="staining_z3",
    threshold_intersection_contain=0.95,
    threshold_intersection_intersect= 1,
    accepted_nb_nuclei_per_cell=None,
    max_cell_nb_intersecting_nuclei=1,
)

Resolving conflicts: 7688it [00:00, 11590.99it/s]


In [5]:
sdata 

SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      └── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
with coordinate systems:
    ▸ 'microns', with elements:
        staining_z3 (Images), transcripts (Points), Cellbound1 (Shapes), Cellbound1_consistent_with_nuclei (Sha

In [6]:
"Cellbound1_consistent_with_nuclei" == "Cellbound1_consistent_with_nuclei"

True